# Overview

Recall for the $k$-dollar graph, there is exactly one assignment up to translation. We can use this fact to construct any genus $2$ graph with $k_1=1,k_2=1,k_3$ from a genus $1$ necklace with $k_3$. In particular for the two trees with the same missing edge, the assignment on the trees is the same. Therefore we can inherit the assignments from the corresponding the weak stability of the genus $1$ graph.

This idea extends to any necklace graph where for a given edge $\{v,w\}$ we take $k$ edges joining $\{v,w\}$.

## Remarks

In particular given a cycle we can can return the weak stability condition.

We can then investigate the phi that work wrt the cycle in phi_analysis.

## Notes

Where graphs must have vertex 0 up to n-1

We store all ouptuts in the appropriate folder of \Graph\graph_stability_conditions as a text file.

We call these graphs GNkM1 where N is the size of the necklace and k is the number of multedges and M1 is where the multedge occurs "v0"-"v1".

Later want to investigate phi for these wsc.

Desktop/For github/Stability_Conditions/Graph/phi_investigation/examples


# Table of contents
1. [Functions](#s1)
    1. [Genus 1 methods ](#s11)
    1. [Extending assignments from genus 1 graphs to GNkM1](#s12)
    1. [Constructing wsc's for graphs GNkM1](#s13)
    1. [Given a cycle returns extended wsc for GNkM1](#s14)
    1. [Storing files](#s15)
2. [Weak stability conditions for $k_1=1,k_2=1,k_3$ graphs](#s2)
3. [Calc cases: Weak stability conditions for GNkM1 graphs](#s3)
3. [Cycle on $G$ to associted wsc for $GNkM1$](#s4)

# Functions <a name="s1"></a>

## Genus 1 methods <a name="s11"></a>

In [1]:
#For change_to_get_trees, produces stability conditions for genus 1 graph given by a cycle.
def head(n, prev_tree_edge):
    """
    n= number of vertices in I_n
    previous element of the cycle, ie tree_edge is j where e_j ={v_j,v_{j+1}} 
    
    #Also used in chip adding on necklace curve
    """
    head = np.zeros(n)

    if prev_tree_edge == n:
        head[0] = 1
    else:
        head[prev_tree_edge] = 1

    return head
def tail(n, current_tree_edge):
    """
    n= number of vertices in I_n
    tree_edge is j where e_j ={v_j,v_{j+1}}
    i is previous element of the cycle, ie tree_edge
    """
    tail = np.zeros(n)
    # if current_tree_edge==n:
    #     tail[0]=-1
    tail[current_tree_edge - 1] = -1

    return tail
def tail_plus(n, current_tree_edge):  #for chip adding
    """
    n= number of vertices in I_n
    tree_edge is j where e_j ={v_j,v_{j+1}}
    i is previous element of the cycle, ie tree_edge
    """
    tail = np.zeros(n)
    # if current_tree_edge==n:
    #     tail[0]=-1
    tail[current_tree_edge - 1] = 1

    return tail

def cycle_to_assignments(cycle):
    #Main to get assingments from cycle

    n = len(cycle)

    # cycle=(1,3,4,2)
    ass_0 = np.zeros(n)

    M = [(1, ass_0)]  # Memory

    for i in range(1, len(cycle)):

        current_tree_edge = cycle[i]
        tup = M[-1]  #pick last item of M

        vtail = tail(n, current_tree_edge)
        vhead = head(n, tup[0])
        prev_ass = tup[1]

        current_ass_t = prev_ass + vtail + vhead
        M.append((current_tree_edge, current_ass_t))

    # for i in M:
    #     print(i)

    return M
def change_to_get_trees(cycle):
    
    """
    Objective:Given a cycle gets a list of assignments (tree,assignment).
    Input:
    Returns
    """
    
    #return trees where there was previously just numbers indicating edge to be removed.
        # change: [(1, array([0., 0., 0., 0.])), (3, array([ 0.,  1., -1.,  0.])), (2, array([ 0.,  0., -1.,  1.])), (4, array([0., 0., 0., 0.]))]
    # so first term is tree of In
    # cycle=(1,3,2,4)

    data_In=cycle_to_assignments(cycle) #<-----------------
    data_In=sorted(data_In, key=lambda x: x[0]) # reordering to (1,),(2,) ect.. instead of being ordered by cycle.
    
    """
    Wish to order data_In by first term of tuple so that is (1, ),(2,) thus putting trees in order so that when we append to Memory M we have
    M'[counter](0) = M[counter][0] when we are iterating through counters.
    """
    
    n=len(cycle)
    G=graphs.CycleGraph(n)
    
    new_data_In=[]
    for data in data_In:
        #Get tree
        tree_info=data[0]
    #     print(tree_info)
        if tree_info== len(data_In):
            edge_tree=(tree_info-1,0)
        else:
            edge_tree=(tree_info-1,tree_info)
    #     print(edge_tree)
        tree=copy(G)
        tree.delete_edge(edge_tree)

        #Recompile data
        new_data_In.append((tree,data[1]))

    # plot(new_data_In[2][0]) #Have the correct trees.
    
    """
    Example:x=change_to_get_trees(cycle)

    print(cycle)
    for i in x:
        print(i[0].edges())
        
        cycle=(1,3,2)
    x=change_to_get_trees(cycle)
    print([tup[0].edges() for tup in x])
    """
    
    return new_data_In

def get_cycles(n):
    """
    Objective:gets all cycles of length n.
    Input:
    Returns
    """
    lst = list(range(1, n + 1))
    x = permutations(lst, n)
    y = [i for i in x if i[0] == 1]

    #Does:
    # x=permutations([1,2,3,4,5],5)
    # y=[i for i in x if i[0]==1]
    # # print(len(y))
    # for i in y:
    #     print(i)

    return y

## Extending assignments from genus 1 graphs to GNkM1 <a name="s12"></a>

In [2]:
def get_ass_repts(G_ass,trees_GM1):
    
    """
    Objective: Given assignments for genus 1 graph,give multiple assignement on graph with k-multiedge.
    G_ass: Assignment (T,ass) on cyclic graph G (no labeling of edges)
    trees_GM1: list of spanning trees for GM1 graph (with labeling of edges)
    """
    
    repeated_ass=[]      
    for t in trees_GM1:
        for pair in G_ass:
            if t==pair[0]:
                repeated_ass.append((t,pair[1])) # neccesray as trees_GM1 has labels and we want these.
    
    return repeated_ass

## Constructing wsc's for graphs GNkM1 <a name="s13"></a>

In [3]:
# Assignment_datum_for_G for w_stability:
def get_sp_trees(G): #Out: list of edges in spanning tree
    all_trees=[]
    for g in list(G.spanning_trees(labels=True)):
        all_trees.append(g.edges(sort=True, labels=True))
    return all_trees
def list_difference(a, b):
    """
    Objective:Takes the difference of removing the 2nd term from the 1st.
    """
    
    "https://stackoverflow.com/questions/8106227/difference-between-two-lists-with-duplicates-in-python"
    count = Counter(a) # count items in a
    count.subtract(b)  # subtract items that are in b
    diff = []
    for x in a:
        if count[x] > 0:
           count[x] -= 1
           diff.append(x)
    return diff

def chip_adding(graph,tree,ass): #new
    """
    Objective:Chip-adds
    Inputs:
    graph: G_edges graph edges
    tree : spanning tree of graph
    ass : an n=vert(graph) tuple on the spanning tree.
    
    recursion idea: https://stackoverflow.com/questions/53638816/python-library-function-to-re-apply-a-function-to-its-own-output-until-output-re
    """
    
    edges=graph # just want the edges 
    n=len(ass)
    complement=list_difference(graph,tree) #edges in complement.
    
    def rec_funct(edge_l,inputs,n):
        data=[]
        edge=edge_l[0]
        
        for b in inputs:
            delta_1=np.zeros(n)
            delta_2=np.zeros(n)
            
            delta_1[int(edge[0])]=1 #we chip add at the first vertex. #### <- CHANGED as taking vertices 0,..,n-1 now
            delta_2[int(edge[1])]=1 #we chip add at the second vertex.
            
            b1=b+delta_1
            b2=b+delta_2
            data=data+[b1,b2]
        
        return (edge_l,data,n)
    
    def recursion(edge_l,inputs,n):
        
        new_data=rec_funct(edge_l,inputs,n)
        edge_l=edge_l[1:] #removeing first edge        
        new_data=(edge_l,new_data[1],new_data[2])
                
        if len(edge_l)==0:
            new_inputs=new_data[1]
            return new_inputs
        else:
            return recursion(*new_data)
        
    breaks=recursion(complement,[np.zeros(n)],n)
    breaks=np.unique(breaks, axis=0)
    
    patch=[]
    for bbreak in breaks:
        patch.append(ass+bbreak)
    
    return patch

def w_stability(graph,Assignments): #Newest
    
    """
    Objective:Takes the chip-adding saturation of the assignments.
    
    #Graphs are labelled v0 to v_n-1
    Inputs:
        graph: Any smallish finite multigraph
        
        Assignments= list of (T,ass) tree and assignment pairs.
    
    Outputs: a list of lists of length vert(graph) corresponding to line bundle multidegrees obtained by chip adding.
    """
    G=graph
    
    G_edges=G.edges(sort=True, labels=True)
    
    lbm_patches=[] # the set of patches,which we'll take the union of. 

    for pair in Assignments:
        tree_edges=pair[0].edges()
        ass=pair[1]    
        patch=chip_adding(G_edges,tree_edges,ass)              
        lbm_patches.append(patch)
    
    sig=np.concatenate(lbm_patches, axis=0)
    sig=sig.astype('int32')    
    sig=np.unique(sig, axis=0)

    return sig

def extend_wsc(Ge,old_wsc=None,cycle_ass=False):
            
    """
    Objective: Turns a wsc for a genus graph into the associated wsc for genus "1" graph with k multiedge  
    Input:
    
    Ge=labeled edge graph  genus "1" graph with k multiedge i.e.
    Ge=Graph([("0","1","e11"),("1","2","e2"),("2","0","e3"),("0","1","e12")], multiedges=True)
    
    old_wsc is a weak stabiltiy condition where
    
    wsc= 
    ([[0, 0, 0], [0, 0, 0], [0, 0, 0]],
     array([[0, 0, 1],
            [0, 1, 0],
            [1, 0, 0]], dtype=int32),
     [(Graph on 3 vertices, [0, 0, 0]),
      (Graph on 3 vertices, [0, 0, 0]),
      (Graph on 3 vertices, [0, 0, 0])],
     [(Multi-graph on 3 vertices, (1, 3, 2), Multi-graph on 3 vertices)])
    
    Returns a the approprate wsc for the graph with k multiedge.
    """
    
    spanning_trees_e=[tree for tree in Ge.spanning_trees(labels=True)]
    
    
    #Assignments only

    if cycle_ass!=False:
        old_ass=cycle_ass
    else:    
        old_ass=old_wsc[2] #with tree graphs (T,div) #!! DONT have trees yet.
        
#     print("cycle_ass",cycle_ass)
#     print(old_ass)

    #Get assignments with trees
    assignments=get_ass_repts(old_ass,spanning_trees_e) #old ass doesnt have labels correct get_ass_repts
    
#     print("assignments",assignments)
    
    lbm=w_stability(Ge,assignments)

    ass_l=[p[1] for p in assignments]
    
    data=[ass_l,lbm,assignments]
    
    return data

def extend_wsc_all(Ge,old_data_l):
    
    """
    Objective: Turns all wsc for a genus graph into the associated wsc for genus "1" graph with k multiedge  

    Input: old_data_l:A list of wsc for the genus 1 graph.
    
    Ge=labeled edge graph  genus "1" graph with k multiedge i.e.
    Ge=Graph([("0","1","e11"),("1","2","e2"),("2","0","e3"),("0","1","e12")], multiedges=True)
    
    Returns: a list of wsc
    """ 
    
    new_data=[]
    for old_wsc in old_data_l:
        new_data.append(extend_wsc(Ge,old_wsc))
    return new_data

In [4]:
# l = [get_sp_trees,extend_wsc,chip_adding,list_difference,extend_wsc_all]
# find_functions_used_l(l,True)

## Given a cycle returns extended wsc for GNkM1 <a name="s14"></a>

In [5]:
def extend_cycle_wsc(Ge,cycle):

    """
    Objective: Turns the cycle wsc for a genus graph into the associated wsc for genus "1" graph with k multiedge  
    Input:
    
    cycle=(1,3,2)

    
    Ge=labeled edge graph  genus "1" graph with k multiedge i.e.
    Ge=Graph([("0","1","e11"),("1","2","e2"),("2","0","e3"),("0","1","e12")], multiedges=True)
        
    Note: Builds on extend_wsc
    
    Returns a the approprate wsc for the graph with k multiedge.
    """

    cycle_ass=change_to_get_trees(cycle) #assighnments (Tree,ass) form

    n=len(cycle)
    relabeler={i: str(i) for i in range(n)}

    for pair in cycle_ass:

        tree=pair[0]
        tree.relabel(relabeler)

    # tree_ex=cycle_ass[0][0]
    # t=tree_ex.edges()  #the vertices are ins we want string format.
    # t

    cycle_ass=[(pair[0],list(pair[1].astype(int))) for pair in cycle_ass]

    data=extend_wsc(Ge,None,cycle_ass)

    return data

## Storing files <a name="s15"></a>

In [13]:
def output_to_file(G,data):#in text files
    
    """
    Objective:prints the data to a text file.
    Input:data is a list of e.g.
    
        [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
     array([[0, 1, 1],
            [0, 2, 0],
            [1, 0, 1],
            [1, 1, 0],
            [2, 0, 0]], dtype=int32),
     [(Graph on 3 vertices, [0, 0, 0]),
      (Graph on 3 vertices, [0, 0, 0]),
      (Graph on 3 vertices, [0, 0, 0]),
      (Graph on 3 vertices, [0, 0, 0]),
      (Graph on 3 vertices, [0, 0, 0])]]
    
    Returns
    """
    
    
    ordering_trees=[t for t in G.spanning_trees(labels=True)]


    print(f"Information: Here are all stability conditions for the graph G")
    print(f"Graph: {G.edges()} \n")
    
    print(f"There are {len(data)} stability conditions for G. \n")
    print(f"The spanning trees of G are ordered as follows:")
    for index,tree in enumerate(ordering_trees):
        print(f"Tree {index+1}: {tree.edges()}" )

    print("\n We will now list all stabilty conditions. The data will be presented in two lines:")
    print("   1. The assignment datum on trees the order of which will be given by the Tree order,")
    print("   2. The union of sets of the form assignment datum of T + the set of break divisors for T. \n" )          

    for wsc in data:
        print(wsc[0]) #assignment datum
        #Put sigma to list format
        print(wsc[1].tolist()) #lbm
        print("\n")
    
    return

def store_data(G,data,graphname="test"):
    name=f"genus_2_graphs_with_middle_edges/GNkM1/{graphname}.txt"
    with open(name, "w") as f:
        sys.stdout = f
        output_to_file(G,data)
        sys.stdout = sys.__stdout__
    return



def pickle_data(graphname,data):
    # Pickle the list
    filename=f"genus_2_graphs_with_middle_edges/GNkM1/{graphname}.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    return

def unpickle(graphname):
    # Unpickle the list
#     filename=f"../find_cycles/examples/{graphname}/{graphname}.pkl"
    
    filename=f"../find_cycles/Done_examples/{graphname}/{graphname}.pkl"
    with open(filename, 'rb') as f:
        unpickled_list = pickle.load(f)
    return unpickled_list

# Calc cases: Weak stability conditions for $k_1=1,k_2=1,k_3$ graphs <a name="s2"></a>

In [14]:
graphname="G3"
G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)
old_data_l=unpickle(graphname) #Size=2

extend_graphname="G3M1"
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","0","e3"),("0","1","e12")], multiedges=True)

data=extend_wsc_all(Ge,old_data_l)

#store pkl file
pickle_data(extend_graphname,data)

#store text file
store_data(Ge,data,graphname=extend_graphname)

In [15]:
graphname="G4"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","0")], multiedges=True)
old_data_l=unpickle(graphname) #Size=2

extend_graphname="G4M1" 
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","0","e4"),("0","1","e12")], multiedges=True)

data=extend_wsc_all(Ge,old_data_l)

#store pkl file
pickle_data(extend_graphname,data)

#store text file
store_data(Ge,data,graphname=extend_graphname)

In [16]:
graphname="G5"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0")], multiedges=True)
old_data_l=unpickle(graphname) #Size=2

extend_graphname="G5M1" 
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","4","e4"),("4","0","e5"),("0","1","e12")], multiedges=True)

data=extend_wsc_all(Ge,old_data_l)

#store pkl file
pickle_data(extend_graphname,data)

#store text file
store_data(Ge,data,graphname=extend_graphname)

In [17]:
graphname="G6"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0")], multiedges=True)
old_data_l=unpickle(graphname) #Size=120

extend_graphname="G6M1" 
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","4","e4"),("4","5","e5"),("5","0","e6"),("0","1","e12")], multiedges=True)

data=extend_wsc_all(Ge,old_data_l)

#store pkl file
pickle_data(extend_graphname,data)

#store text file
store_data(Ge,data,graphname=extend_graphname)

# Calc cases: Weak stability conditions for GNkM1 graphs <a name="s3"></a>

In [24]:
#"G33M1"
graphname="G3"
G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)
old_data_l=unpickle(graphname) #Size=2

extend_graphname="G33M1"
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","0","e3"),("0","1","e12"),("0","1","e13")], multiedges=True)

data=extend_wsc_all(Ge,old_data_l)

#store pkl file
pickle_data(extend_graphname,data)

#store text file
store_data(Ge,data,graphname=extend_graphname)

In [21]:
#"G34M1"

extend_graphname="G34M1"
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","0","e3"),("0","1","e12"),("0","1","e13"),("0","1","e14")], multiedges=True)

data=extend_wsc_all(Ge,old_data_l)

#store pkl file
pickle_data(extend_graphname,data)

#store text file
store_data(Ge,data,graphname=extend_graphname)

In [22]:
graphname="G4"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","0")], multiedges=True)
old_data_l=unpickle(graphname) #Size=2

extend_graphname="G43M1" 
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","0","e4"),("0","1","e12"),("0","1","e13")], multiedges=True)

data=extend_wsc_all(Ge,old_data_l)

#store pkl file
pickle_data(extend_graphname,data)

#store text file
store_data(Ge,data,graphname=extend_graphname)

In [23]:
extend_graphname="G44M1" 
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","0","e4"),("0","1","e12"),("0","1","e13"),("0","1","e14")], multiedges=True)

data=extend_wsc_all(Ge,old_data_l)

#store pkl file
pickle_data(extend_graphname,data)

#store text file
store_data(Ge,data,graphname=extend_graphname)

# Cycle on $G$ to associted wsc for $GNkM1$ <a name="s4"></a>

## G3

Here we take the data for the cycle (1,3,2) for the graphs, G3,G3M1,G33M1,G34M1 in ordder to analyse the effect on phi.

In [26]:
cycle=(1,3,2)

In [27]:
graphname="G3"
G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)
old_data_l=unpickle(graphname) #Size=2

In [28]:
#G3
d1=extend_cycle_wsc(G,cycle)

In [29]:
# extend_graphname="G3M1"
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","0","e3"),("0","1","e12")], multiedges=True)
d2=extend_cycle_wsc(Ge,cycle)

In [30]:
# extend_graphname="G33M1"
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","0","e3"),("0","1","e12"),("0","1","e13")], multiedges=True)
d3=extend_cycle_wsc(Ge,cycle)

In [31]:
# extend_graphname="G34M1"
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","0","e3"),("0","1","e12"),("0","1","e13"),("0","1","e14")], multiedges=True)
d4=extend_cycle_wsc(Ge,cycle)

In [33]:
data=[d1,d2,d3,d4]
# data

In [34]:
#store pkl file
pickle_data("132_G3kM1",data)

# G4

In [35]:
cycle=(1,3,4,2)

In [36]:
graphname="G4"
G=Graph([("0","1"),("1","2"),("2","3"),("3","0")], multiedges=True)
old_data_l=unpickle(graphname) #Size=6

In [37]:
#G4
d1=extend_cycle_wsc(G,cycle)

In [38]:
# extend_graphname="G4M1"
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","0","e4"),("0","1","e12")], multiedges=True)
d2=extend_cycle_wsc(Ge,cycle)

In [39]:
# extend_graphname="G43M1" 
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","0","e4"),("0","1","e12"),("0","1","e13")], multiedges=True)

d3=extend_cycle_wsc(Ge,cycle)

In [40]:
# extend_graphname="G44M1" 
Ge=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","0","e4"),("0","1","e12"),("0","1","e13"),("0","1","e14")], multiedges=True)
d4=extend_cycle_wsc(Ge,cycle)

In [42]:
data=[d1,d2,d3,d4] # num of assignmets 4,7,10,13
# data

In [43]:
#store pkl file
pickle_data("1342_G4kM1",data)

# Used Throughout

In [8]:
import numpy as np
import networkx as nx
from collections import Counter
import itertools 
import random
from itertools import combinations, permutations
import sys
import pickle

In [9]:
import ast
import inspect

In [10]:
def find_functions_used(func):
    
    """Add print Objective."""
    
    functions_used = set()
    tree = ast.parse(inspect.getsource(func))
    for node in ast.walk(tree):
        if isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
            functions_used.add(node.func.id)
    return functions_used

def extract_objective(func):
    """
    Extracts the string of the line containing "Objective" from the docstring of a given function.

    Args:
        func: The function to extract the objective from.

    Returns:
        A string containing the objective.
    """
    
    """
        def f():

        
            Objective: This is a test
            Input:
            Return
        

        x=33

        return
        extract_objective(f)
    """
    docstring = inspect.getdoc(func)
    if docstring is None:
        return None
    lines = docstring.split('\n')
    for line in lines:
        if 'Objective' in line:
            return line.strip()
    return None


def find_functions_used_l(l,flag=False):
    for func in l:
        functions_used = find_functions_used(func)
        if flag==True:
            print(f"{func.__name__}:\n{extract_objective(func)}\n{functions_used} \n")
        if flag==False:
            print(f"{func.__name__}:\n{functions_used} \n")

In [ ]:
# l = [f1, f2, f3]
# find_functions_used_l(l,True)